# Zillow Clustering Project

## Agenda

## Executive Summary

<div class="alert alert-block alert-info"><b>Finish the mvp workflow and then go back and think critically about what you might of missed. The point isn't to doubt yourself; the purpose here is to check your blind spots and see if you can't find more information or insights that will help you to deliver better results.

Better yet, and this cannot be stressed enough, don't ask yourself these questions, but bring them up in conversation with peers, experts in other fields, or even complete strangers -- anyone with a different point of view is going to be able to help you to see what things you are taking for granted</div>



## Description: 
- Zillow: What is driving the errors in the Zestimates?
- The presentation will consist of a notebook demo of the discoveries you made and work you have done related to uncovering what the drivers of the error in the zestimate is.


## Problem Statement
- What is driving the error in the Zestimate (logerror)?
- Are there features that will produce a smaller average error?
- What groups are in the data?
- Does using these groups help remove the noise in the model?

#### Domain Research
- What is a single unit housing unit?
    - https://www.investopedia.com/terms/h/housingunits.asp
- What is fips?
    - https://en.wikipedia.org/wiki/FIPS_county_code
- What is the min/max tax rate by county in US?
    - https://www.attomdata.com/news/market-trends/figuresfriday/top-10-u-s-counties-with-the-greatest-effective-tax-rates/
- Understanding Zillow Zestimate
    - https://www.zillow.com/zestimate/
    - https://www.youtube.com/watch?v=rfWzMI_VwTQ
    - https://www.kaggle.com/c/zillow-prize-1/overview

<div class="alert alert-block alert-info"><b>

## PLAN

**MVP Questions to answer:**
- Use clustering to identify new features
- Determine the baseline logerror
- Produce a model that has less logerror than baseline
- Determine key drivers of error

**Brainstorm:**    
- MVP all counties
- features to add:
    - county name column, remove fips, get dummies for clustering use
    - combined bedroom/bathroom ratio? drop bathroom, keep bedroom
    - cluster on size with lot size, finished sq ft, and ????
    - convert year build to age (current date - yearbuilt)
    - does age correlate with size? if so maybe cluster age with size columns above
    - cluster on tax value? taxamount, taxvaluedollarcnt, landtaxvaluedollarcnt, structuretaxvaluedollarcnt?
- 2nd run add tax percent rate column and remove outliers above 6.6% and below 1% 
    - first outlier method removed too many observations, this was done instead
    
- 2nd run through seperate models by county?


Audience: Zillow Data Science Team    
Setting: Professional


<div class="alert alert-block alert-info"><b>

## Acquire
DELIVERABLES: 
- Data is collected from the Codeup cloud database with an appropriate SQL query
- Define single unit property

Summary of query requirements and creation:



****
Acquire    
**take aways**
1. could fireplace, garage, pool, hottub, deck be made to 0 or 1 then summed as "plus_item" column?
    - this would assume null values do not have the feature, as opposed to feature is present but not noted
2. drop features with 70% or more missing values to start
3. most rows have 32-34 columns with missing values
****

<div class="alert alert-block alert-info"><b>Prep needs additional summary on handling nulls and removing outliers</div>

In [ ]:
import wrangle_zillow

df, X_train, y_train, X_validate, y_validate, X_test, y_test, X_train_scaled, X_validate_scaled, X_test_scaled = wrangle_zillow.wrangle_zillow_cluster()
# prints shape of X and X scaled dataframess

## Preparation
DELIVERABLES:
- Column data types are appropriate for the data they contain
- Missing values are investigated and handled
- Outliers are investigated and handled

Summary of handling nulls, outliers, creating additional features, dropping columns/rows

Data split into X (features) and y (target) dataframes for Train, Validate, and Test
Data for X dataframes scaled for cluster exploration and modeling


In [ ]:
# During this stage called this summary function to assist in seeing remain nulls to handle and
# for histograms of individual variables.
# Can be used for full dataframe or any of the split or scaled dataframes
import summarize

fulldf = summarize.df_summary(df)

****
Prepare     
**take aways**    

**This is deffinition used in previous project**   

Determine deffinition of single property used article by James Chen Updated Sep 11, 2020 What Is a Housing Unit? "The term housing unit refers to a single unit within a larger structure that can be used by an individual or household to eat, sleep, and live. The unit can be in any type of residence such as a house, apartment, mobile home, or may also be a single unit in a group of rooms. Essentially, a housing unit is deemed to be a separate living quarter where the occupants live and eat separately from other residents of the structure or building. They also have direct access from the building's exterior or through a common hallway."
https://www.investopedia.com/terms/h/housingunits.asp   

**In my opinion deffinition should include condo, townhouse, any unit that can be sold to an individual owner. So my deffinition will include townhouse, condo, ect. but not commercial, business, land only, etc. **   

This site has the property use codes for LA county https://www.titleadvantage.com/mdocs/LA%20County%20Use%20Codes%20nm.pdf   
looking at the common use codes for Duplex, Triplex, and Quadplex these codes indicate the units are multi-family/income properties or retail/store properties so these will be excluded

Identify Properties in the Database: Based on the above definition some categories do not fit brief   
Propertylandusetypeid | propertylandusedesc    
No 31 Commercial/Office/Residential Mixed Used (not a residence)    
No 46 Multi-Story Store (not a residence)   
No 47 Store/Office (Mixed Use) (not a residence)    
No 246 Duplex (2 Units, Any Combination)    
No 247 Triplex (3 Units, Any Combination)   
No 248 Quadruplex (4 Units, Any Combination)   
260 Residential General    
261 Single Family Residential   
262 Rural Residence   
263 Mobile Home   
264 Townhouse   
No 265 Cluster Home    
266 Condominium    
No 267 Cooperative (become shareholder not owner)   
268 Row House       
No 269 Planned Unit Development   
No 270 Residential Common Area (propterty feature)    
No 271 Timeshare (become shareholder not owner)    
273 Bungalow      
274 Zero Lot Line   
275 Manufactured, Modular, Prefabricated Homes   
276 Patio Home    
279 Inferred Single Family Residential      
No 290 Vacant Land - General (not a residence)   
No 291 Residential Vacant Land (not a residence)   

So we will keep only those where propertylandusetypeid = ('260', '261', '262', '263', '264', '266', '268', '273', '274', '275', '276', '279')  

**acquire function updated to filter only for these**
new shape = (71789, 70)
****

<div class="alert alert-block alert-info">

<div class="alert alert-block alert-info"><b>Explore needs cleaned up and funcions created and called. Also need to add interaction between indenpendent varialbes and target. Add 4th clustering model on location?</div>

## Exploration
DELIVERABLES:      
**- Interaction between independent variables and the target variable is explored using visualization and statistical testing**
- Clustering is used to explore the data
- A conclusion, supported by statistical testing and visualization, is drawn on whether or not the clusters are helpful/useful
- At least 3 combinations of features for clustering should be tried



In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import explore

In [ ]:
X_train_scaled.columns

cluster on size with lot size, finished sq ft, and ????
does age correlate with size? if so maybe cluster age with size columns above
cluster on tax value? taxamount, taxvaluedollarcnt, landtaxvaluedollarcnt, structuretaxvaluedollarcnt?

#### Does age correlate with size?
1. decided to use calculatedfinishedsquarefeet for size to answer this
2. visualize age and size together (both continuous variables)
3. use scatter plot for chart
4. use stats.pearsonr for hypothesis testing

#### Hypothesis test
Ho: There is no relationship between age and size.     
Ha: There is a relationship between age and size.


In [ ]:
# for final this will need to be a function
plt.figure(figsize=(16, 9))
sns.scatterplot(data=X_train, y='calculatedfinishedsquarefeet', x='age', hue='county', alpha=.6)

Given this visual it may be better to separate by county and retest in next iteration

In [ ]:
# for final this will need to be a function
alpha = .05
x = X_train.age
y = X_train.calculatedfinishedsquarefeet
r, p = stats.pearsonr(x, y)
r, p

The p-value is less than alpha so we reject the null hypothesis, though the r value is not far from zero and is negative, which would indicate a weak negative linear correlation. 

Based on correlation of age and size (though weak) will try clustering on calculatedfinishedsquarefeet, lotsizesquarefeet, and age.    
This produced a very slight improvement over baseline. Removing age from this cluster to see impact.

In [ ]:
cluster_vars = ['calculatedfinishedsquarefeet_scaled', 'lotsizesquarefeet_scaled']
explore.elbow_plot(X_train_scaled, cluster_vars)
# based on this will use 5 as k value

In [ ]:
k=5
cluster_col_name = 'size_cluster'
train_clusters, kmeans = explore.run_kmeans(X_train_scaled, X_train, cluster_vars, k, cluster_col_name)

In [ ]:
train_clusters.size_cluster.value_counts()
# so so dispersal of values

In [ ]:
centroids = explore.get_centroids(kmeans, cluster_vars, cluster_col_name)

In [ ]:
X_train, X_train_scaled = explore.add_to_train(X_train, train_clusters, X_train_scaled, centroids, cluster_col_name)

In [ ]:
val_trans_clusters = explore.kmeans_transform(X_validate_scaled, kmeans, cluster_vars, cluster_col_name)
test_trans_clusters = explore.kmeans_transform(X_test_scaled, kmeans, cluster_vars, cluster_col_name)

In [ ]:
X_validate, X_validate_scaled = explore.add_to_train(X_validate, val_trans_clusters, X_validate_scaled, centroids, cluster_col_name)
X_test, X_test_scaled = explore.add_to_train(X_test, test_trans_clusters, X_test_scaled, centroids, cluster_col_name)

In [ ]:
X_train.columns

In [ ]:
X_train_scaled.columns

In [ ]:
X_validate.columns

In [ ]:
X_test.columns

In [ ]:
# dollar amount clustering
# use structuretaxvaluedollarcnt, taxvaluedollarcnt, taxamount, structure_dollar_per_sqft, land_dollar_per_sqft, taxrate
cluster_vars2 = ['structuretaxvaluedollarcnt_scaled', 'taxvaluedollarcnt_scaled', 'taxamount_scaled', 'structure_dollar_per_sqft_scaled', 'land_dollar_per_sqft_scaled', 'taxrate_scaled']
explore.elbow_plot(X_train_scaled, cluster_vars2)
# based on this will use 5 as k value

In [ ]:
k2=5
cluster_col_name2 = 'dollar_cluster'
train_clusters2, kmeans2 = explore.run_kmeans(X_train_scaled, X_train, cluster_vars2, k2, cluster_col_name2)

In [ ]:
train_clusters2.dollar_cluster.value_counts()
# based on this distribution this clustering may not be that helpful

In [ ]:
centroids2 = explore.get_centroids(kmeans2, cluster_vars2, cluster_col_name2)

In [ ]:
X_train, X_train_scaled = explore.add_to_train(X_train, train_clusters2, X_train_scaled, centroids2, cluster_col_name2)

#### Does bed/bath ratio correlate with age?
1. visualize bed_bath_ratio and age together (both continuous variables)
2. use scatter plot for chart
3. based on chart bin bed_bath ratio and age and visualize
4. use chi squared test to compare 

#### Hypothesis test
Ho: Bed_bath_ratio and age are independent.     
Ha: Bed_bath_ratio and age are dependent.



In [ ]:
# for final this will need to be a function
plt.figure(figsize=(16, 9))
sns.scatterplot(data=X_train, y='age', x='bed_bath_ratio', hue='county', alpha=.6)

In [ ]:
# create binned bed_bath_ratio column
X_train['bb_bin'] = pd.cut(X_train.bed_bath_ratio, bins=[0, 1, 2, 3, 4], labels=['1', '2', '3', '4+'])
# create binned age column by 20 yr groups
X_train['age_bin'] = pd.cut(X_train.age, bins=[0, 20, 40, 60, 80], labels=['<20', '40', '60', '80+'])

In [ ]:
# visualize these 2 categorical variables together
# Heatmap of bined values (Categorical and Categorical)
plt.figure(figsize=(16,9))
ctab = pd.crosstab(X_train.bb_bin, X_train.age_bin, normalize=True)
sns.heatmap(ctab, cmap='Purples', annot=True, fmt='.1%')
plt.title('What is correlation between bined bed/bath ratio and binned age of home?')
plt.show()

In [ ]:
# Hypothesis Test

alpha = 0.05
contingency_table = pd.crosstab(X_train.bb_bin, X_train.age_bin)

chi2, p, degf, expected = stats.chi2_contingency(contingency_table)

if p < alpha:
    print("We reject the null hypothesis")
    print(f'chi^2 = {chi2:.4f}')
    print(f'p     = {p:.4f}')
else:
    print("We fail to reject the null hypothesis")

In [ ]:
X_train_scaled.columns

In [ ]:
# create rooms cluster
# use bedroomcnt, fullbathcnt, roomcnt, structure_dollar_per_sqft, land_dollar_per_sqft, taxrate
cluster_vars3 = ['bedroomcnt_scaled', 'fullbathcnt_scaled', 'roomcnt_scaled', 'bed_bath_ratio_scaled']
explore.elbow_plot(X_train_scaled, cluster_vars3)
# based on this will use 6 as k value

In [ ]:
k3=6
cluster_col_name3 = 'rooms_cluster'
train_clusters3, kmeans3 = explore.run_kmeans(X_train_scaled, X_train, cluster_vars3, k3, cluster_col_name3)

In [ ]:
train_clusters3.rooms_cluster.value_counts()
# pretty good dispersal of values

In [ ]:
centroids3 = explore.get_centroids(kmeans3, cluster_vars3, cluster_col_name3)

In [ ]:
X_train, X_train_scaled = explore.add_to_train(X_train, train_clusters3, X_train_scaled, centroids3, cluster_col_name3)

In [ ]:
# 2nd iteration: add area/location cluster with longitude, latitude, encoded counties?
X_train_scaled.columns

In [ ]:
# create location cluster
# use longitude, latitude, age
cluster_vars4 = ['longitude_scaled', 'latitude_scaled', 'age_scaled']
explore.elbow_plot(X_train_scaled, cluster_vars4)
# based on this will use 6 as k value

In [ ]:
k4=6
cluster_col_name4 = 'loc_cluster'
train_clusters4, kmeans4 = explore.run_kmeans(X_train_scaled, X_train, cluster_vars4, k4, cluster_col_name4)

In [ ]:
train_clusters4.loc_cluster.value_counts()
# good dispersal of values

In [ ]:
centroids4 = explore.get_centroids(kmeans4, cluster_vars4, cluster_col_name4)

In [ ]:
X_train, X_train_scaled = explore.add_to_train(X_train, train_clusters4, X_train_scaled, centroids4, cluster_col_name4)

<div class="alert alert-block alert-info"><b>Modeling needs various X_train dataframe creation moved to model.py and then try different types of models?</div>

## Modeling Stage
DELIVERABLES: 
- At least 4 different models are created and their performance is compared
- One model is the distinct combination of algorithm, hyperparameters, and features
- Best practices on data splitting are followed


In [ ]:

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import model
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression, SelectKBest, RFE 
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from math import sqrt
from scipy import stats

In [ ]:
X_train_scaled.columns

In [ ]:
# try a lineat regression model with original (not clustered) features
XtrainSO = X_train_scaled.drop(columns=['size_cluster',
       'centroid_calculatedfinishedsquarefeet_scaled',
       'centroid_lotsizesquarefeet_scaled', 'dollar_cluster',
       'centroid_structuretaxvaluedollarcnt_scaled',
       'centroid_taxvaluedollarcnt_scaled', 'centroid_taxamount_scaled',
       'centroid_structure_dollar_per_sqft_scaled',
       'centroid_land_dollar_per_sqft_scaled', 'centroid_taxrate_scaled',
       'rooms_cluster', 'centroid_bedroomcnt_scaled',
       'centroid_fullbathcnt_scaled', 'centroid_roomcnt_scaled',
       'centroid_bed_bath_ratio_scaled', 'loc_cluster',
       'centroid_longitude_scaled', 'centroid_latitude_scaled',
       'centroid_age_scaled'])
XtrainSO.head()

In [ ]:
# try a lineat regression model with original and size cluster features
XtrainSS = X_train_scaled.drop(columns=['dollar_cluster',
       'centroid_structuretaxvaluedollarcnt_scaled',
       'centroid_taxvaluedollarcnt_scaled', 'centroid_taxamount_scaled',
       'centroid_structure_dollar_per_sqft_scaled',
       'centroid_land_dollar_per_sqft_scaled', 'centroid_taxrate_scaled',
       'rooms_cluster', 'centroid_bedroomcnt_scaled',
       'centroid_fullbathcnt_scaled', 'centroid_roomcnt_scaled',
       'centroid_bed_bath_ratio_scaled', 'loc_cluster',
       'centroid_longitude_scaled', 'centroid_latitude_scaled',
       'centroid_age_scaled'])
XtrainSS.head()

In [ ]:
# try a lineat regression model with original and dollar cluster features
XtrainSD = X_train_scaled.drop(columns=['size_cluster',
       'centroid_calculatedfinishedsquarefeet_scaled',
       'centroid_lotsizesquarefeet_scaled', 'rooms_cluster', 'centroid_bedroomcnt_scaled',
       'centroid_fullbathcnt_scaled', 'centroid_roomcnt_scaled',
       'centroid_bed_bath_ratio_scaled', 'loc_cluster',
       'centroid_longitude_scaled', 'centroid_latitude_scaled',
       'centroid_age_scaled'])
XtrainSD.head()

In [ ]:
# try a lineat regression model with original and room cluster features
XtrainSR = X_train_scaled.drop(columns=['size_cluster',
       'centroid_calculatedfinishedsquarefeet_scaled',
       'centroid_lotsizesquarefeet_scaled', 'dollar_cluster',
       'centroid_structuretaxvaluedollarcnt_scaled',
       'centroid_taxvaluedollarcnt_scaled', 'centroid_taxamount_scaled',
       'centroid_structure_dollar_per_sqft_scaled',
       'centroid_land_dollar_per_sqft_scaled', 'centroid_taxrate_scaled',
       'loc_cluster','centroid_longitude_scaled', 'centroid_latitude_scaled',
       'centroid_age_scaled'])
XtrainSR.head()

In [ ]:
# try a lineat regression model with original and location cluster features
XtrainSL = X_train_scaled.drop(columns=['size_cluster',
       'centroid_calculatedfinishedsquarefeet_scaled',
       'centroid_lotsizesquarefeet_scaled', 'dollar_cluster',
       'centroid_structuretaxvaluedollarcnt_scaled',
       'centroid_taxvaluedollarcnt_scaled', 'centroid_taxamount_scaled',
       'centroid_structure_dollar_per_sqft_scaled',
       'centroid_land_dollar_per_sqft_scaled', 'centroid_taxrate_scaled',
       'rooms_cluster', 'centroid_bedroomcnt_scaled',
       'centroid_fullbathcnt_scaled', 'centroid_roomcnt_scaled',
       'centroid_bed_bath_ratio_scaled'])
XtrainSL.head()

In [ ]:
# now a model with all features and clusters
XtrainSALL = X_train_scaled.copy()
XtrainSALL.head()

In [ ]:
# and a model with only clusters
XtrainSOC = X_train_scaled.drop(columns=['bedroomcnt_scaled', 'calculatedfinishedsquarefeet_scaled',
       'fullbathcnt_scaled', 'lotsizesquarefeet_scaled', 'roomcnt_scaled',
       'unitcnt_scaled', 'structuretaxvaluedollarcnt_scaled',
       'taxvaluedollarcnt_scaled', 'taxamount_scaled', 'age_scaled',
       'taxrate_scaled', 'structure_dollar_per_sqft_scaled',
       'land_dollar_per_sqft_scaled', 'bed_bath_ratio_scaled'])

### Train models
***for MVP these are variations on Linear Regression Model altering features**

In [ ]:
baseline = model.get_baseline(y_train, y_train)

In [ ]:
# Scaled, original features only
RMSE_m1SO = model.linear_reg_model(XtrainSO, y_train)
RMSE_m1SO
# slightly better than baseline

In [ ]:
# Scaled, original with size cluster only
RMSE_m2SS = model.linear_reg_model(XtrainSS, y_train)
RMSE_m2SS
# slightly better than original features only

In [ ]:
# Scaled, original with dollar cluster only
RMSE_m3SD = model.linear_reg_model(XtrainSD, y_train)
RMSE_m3SD
# slightly better than size cluster 2nd

In [ ]:
# Scaled, original with room cluster only
RMSE_m4SR = model.linear_reg_model(XtrainSR, y_train)
RMSE_m4SR
# about the same as dollar cluster, 3rd

In [ ]:
# Scaled, original with location cluster only
RMSE_m5SL = model.linear_reg_model(XtrainSL, y_train)
RMSE_m5SL
# about the same as size cluster

In [ ]:
# Scaled, original with ALL clusters
RMSE_m6SALL = model.linear_reg_model(XtrainSALL, y_train)
RMSE_m6SALL
# best of these, 1st

In [ ]:
# Scaled, only clusters
RMSE_m7SOC = model.linear_reg_model(XtrainSOC, y_train)
RMSE_m7SOC
# barely better than baseline and worse than with no clusters

******
**Take aways from Linear Regression models**     
Model with all original features and clusters performed best. Though not much better than baseline.

**Next Step**     
- try alternate algorithm
- create loop that will summarize and display results
*****

In [ ]:
df2test = [XtrainSO, XtrainSS, XtrainSD, XtrainSR, XtrainSL, XtrainSALL, XtrainSOC]
target = y_train

In [ ]:
# this doesn't work
# for df in df2test:
#     RMSE = model.lasso_lars(df, target)
#     print(RMSE)
    


In [ ]:
XtrainSO.head()

In [ ]:
def lasso_lars(x_scaleddf, target):
    # Make a model
    lars = LassoLars(alpha=1)
    # Fit a model
    lars.fit(x_scaleddf, target)
    # Make Predictions
    lars_pred = lars.predict(x_scaleddf)
    # Computer root mean squared error
    lars_rmse = sqrt(mean_squared_error(target, lars_pred))
    return lars_rmse

In [ ]:
RMSE_m1SO = lasso_lars(XtrainSO, y_train)
RMSE_m1SO

In [ ]:
XtrainSS.head()

In [ ]:
RMSE_m2SS = lasso_lars(XtrainSS, y_train)
RMSE_m2SS

In [ ]:
RMSE_m3SALL = lasso_lars(XtrainSO, y_train)
RMSE_m3SALL

### Validate

### Test

<div class="alert alert-block alert-info"><b>

## Delivery Stage
DELIVERABLES:
- [Main_Notebook](xxxxxxx)
- walk through of notebook



### Summary of findings



### Top perfoming model



### Next Steps

<div class="alert alert-block alert-info"><b>

# Notes: place in correct section or remove?

determine and handle upper and lower outliers    
While this area of the country does have a broad range of housing values and therefore tax values, we know from previous work with this dataset that the tax rate range is from .001% up to 45% which is unrealistic for tax rates in 2017. Therefore we will determine a point at which to drop outliers above and below the IQR.

*****
MVP - used taxamount to determine cutoff for outliers    
for 2nd round might add calculated tax rate column and use that for cut off instead.  
**Upperbound Outliers to HANDLE**   

**taxamount_outliers**     
count      4894.000000     
mean      10979.609790     
std       20235.975915     
min           0.830000     
25%        1997.677500     
50%        5367.625000     
75%       12289.405000     
max      573508.600000     
Name: taxamount_outliers, dtype: float64     
    
*******
**based on the above info drop upperbound outliers that are above 50th percentile of upperbound outliers**

reasoning: there is a significant jump from the 25th to 50th percentile making this a logical place to start removing outliers while still keeping some outliers above the upperbound
******

********
**Lowerbound Outliers to HANDLE**

**taxamount_outliers**     
count    68981.000000    
mean      8203.408061    
std       3350.880803    
min       3210.485000    
25%       5817.075000    
50%       7473.745000    
75%       9695.125000    
max      21685.205000    
Name: taxamount_low_outliers, dtype: float64                  
   
*******
**based on the above drop rows that are above the 75th lowerbound percentile**

reasoning: there is a significant jump from the 50th to 75th percentile making this a logical place to start removing outliers while still keeping some outliers above the lowerbound
*****